<p align="center">
<FONT FACE="times new roman" SIZE=5>
<br>
<i><b>Docente:</b> Ricardo Ándres Fonseca Perdomo.</i>
<br>
<i><b>Asignatura:</b> Aprendizaje de máquina</i>
<br>
<i><b>Estudiante:</b> Chiara Valenzuela</i>
<br>
<i><b>Tema:</b> Trabajo en clase</i>
<br>
<i>20/03/24</i>
</FONT>
</p>


### Importaciones

In [ ]:
import sys
assert sys.version_info >= (3, 7)

from packaging import version
import sklearn
assert version.parse(sklearn.__version__) >= version.parse("1.0.1")

from pathlib import Path
import pandas as pd
import tarfile
import urllib.request
import matplotlib.pyplot as plt
import numpy as np
from zlib import crc32
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
from scipy.stats import binom
from pandas.plotting import scatter_matrix
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder

## Clase 1

In [ ]:
def load_dataset_url():
    tarball_path = Path('datasets/housing.tgz')
    if not tarball_path.is_file():
        Path("datasets").mkdir(parents=True,exist_ok=True)
        url = 'https://github.com/ageron/data/raw/main/housing.tgz'
        urllib.request.urlretrieve(url, tarball_path)
        with tarfile.open(tarball_path) as housing_tarball:
            housing_tarball.extractall(path="datasets")
    return pd.read_csv("datasets/housing/housing.csv")

dt = load_dataset_url()

In [ ]:
dt.info()

In [ ]:
dt['ocean_proximity'].value_counts()

In [ ]:
dt.describe()

In [ ]:
plt.rc('font', size = 12)
plt.rc('axes', labelsize = 12, titlesize = 12)
plt.rc('legend', fontsize = 14)
plt.rc('xtick', labelsize = 10)
plt.rc('ytick', labelsize = 10)

In [ ]:
IMAGES_PATH = Path() / "images" / "images_bar_housing"
IMAGES_PATH.mkdir(parents=True, exist_ok= True)

def save_fig(fig_id, tight_layout = True, fig_extension = "jpg", resolution = 300):
    path =  IMAGES_PATH / f"{fig_id}.{fig_extension}"
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format = fig_extension, dpi = resolution)

In [ ]:
dt.hist(bins = 100, figsize = (15, 7))
save_fig("Grafica atributos housing (Histograma)")
plt.show()

In [ ]:
def shuffle_and_split(dt, test_radio):
    shuffle_indices = np.random.permutation(len(dt))
    test_set_size = int(len(dt) * test_radio)
    test_indices = shuffle_indices[:test_set_size]
    train_indices = shuffle_indices[test_set_size:]
    return dt.iloc[train_indices], dt.iloc[test_indices]

In [ ]:
train_set, test_set = shuffle_and_split(dt, 0.3)

In [ ]:
print(len(train_set), len(test_set))

In [ ]:
np.random.seed(42)

In [ ]:
def is_id_in_test(identifier, test_radio):
    return crc32(np.int64(identifier)) < test_radio * 2**32

In [ ]:
def split_data_with_id_hash(dt, test_radio, id_column):
    ids = dt[id_column]
    in_test_set = ids.apply(lambda id_: is_id_in_test(id_, test_radio))
    return dt.loc[~in_test_set], dt.loc[in_test_set]

In [ ]:
data_with_id = dt.reset_index()
train_set, test_set = split_data_with_id_hash(data_with_id, 0.3, "index")

In [ ]:
data_with_id["id"] = dt['longitude']*1000 + dt['latitude']
train_set, test_set = split_data_with_id_hash(data_with_id, 0.3, "id")

### Ejercicio

<p>
Muestre los resultados de las 2 celdas anteriores y concluya como s cambiaron los datos de entrenamiento y test.
</p>

Después de dividir los datos utilizando diferentes métodos de identificación, los conjuntos de datos de entrenamiento y prueba cambian.

En el primer método, donde se utiliza el índice como identificador, la división es aleatoria y no hay relación entre las instancias y su identificador. Por lo tanto, el histograma del conjunto de datos de entrenamiento no muestra ningún patrón particular.

En el segundo conjunto de operaciones, se crea un nuevo identificador id basado en la longitud y latitud de las instancias. Luego, se aplica split_data_with_id_hash utilizando este nuevo identificador. Esto puede conducir a una división donde las instancias que están geográficamente cerca pueden terminar en el mismo conjunto (ya sea de entrenamiento o prueba). Esta división basada en la ubicación geográfica puede ser útil si se espera que exista alguna correlación espacial en los datos, por ejemplo, si las casas cercanas tienden a tener características similares.

## Clase 2

In [ ]:
train_set, test_set = train_test_split(dt, test_size=0.3, random_state=42)

In [ ]:
train_set.head()

In [ ]:
test_set['total_bedrooms'].isnull().sum()

In [ ]:
sample_size = 1000
ratio_female = 0.511
prob_too_small = binom(sample_size, ratio_female).cdf(485 - 1)
prob_too_large = binom(sample_size, ratio_female).cdf(535)

print(prob_too_large + prob_too_small)

In [ ]:
np.random.seed(42)

In [ ]:
samples = (np.random.rand(100_000, sample_size) < ratio_female).sum(axis=1)
((samples < 485) | (samples > 535)).mean()

In [ ]:
dt['income_category'] = pd.cut(dt['median_income'],
                               bins= [0., 1.5, 3.0, 4.5, 6., np.inf],
                               labels = [1, 2, 3, 4, 5])

In [ ]:
dt.head()

In [ ]:
dt['income_category'].value_counts().sort_index().plot.bar(rot = 0, grid = True)
plt.xlabel('Income category')
save_fig('Income category')
plt.show()

In [ ]:
splitter = StratifiedShuffleSplit(n_splits=10, test_size= 0.3, random_state= 42)
stratified_split = []

for train_index, test_index in splitter.split(dt, dt['income_category']):
    stratified_train_set_n = dt.iloc[train_index]
    stratified_test_set_n = dt.iloc[test_index]
    stratified_split.append([stratified_train_set_n, stratified_test_set_n])

stratified_split[0]

In [ ]:
stratified_train_set, stratified_test_set = stratified_split[0]

In [ ]:
stratified_train_set, stratified_test_set = train_test_split(dt, test_size=0.3, stratify=dt['income_category'] ,random_state=42)

In [ ]:
stratified_test_set['income_category'].value_counts() / len(stratified_test_set)

In [ ]:
dt.plot(kind = 'scatter', x = 'longitude', y = 'latitude', grid = True)
save_fig('Latitud vs Longitud')
plt.show()

In [ ]:
dt.plot(kind = 'scatter', x = 'longitude', y = 'latitude', grid = True, alpha = 0.3)
save_fig('Latitud vs Longitud con alpha')
plt.show()

In [ ]:
dt.plot(kind = 'scatter', x = 'longitude', y = 'latitude', grid = True, s = dt['population']/100, label = 'population', c = 'median_house_value', cmap = 'jet', colorbar = True)
save_fig('Latitud vs Longitud con mapa de calor')
plt.show()

## Clase 3

### Buscando correlaciones

In [ ]:
corr_matrix = dt.corr(numeric_only=True)
corr_matrix

In [ ]:
corr_matrix['median_house_value'].sort_values(ascending = False)

In [ ]:
atributes = ['median_house_value', 'median_income', 'total_rooms',
             'housing_median_age']

scatter_matrix(dt[atributes], figsize=(10,10))
plt.show()

In [ ]:
dt.plot(kind = 'scatter', x = 'median_income', y = 'median_house_value',
        alpha = 0.2, grid = True)
plt.show()

In [ ]:
dt['rooms_per_house'] = dt['total_rooms'] / dt['households']
print(dt['rooms_per_house'].head())
dt['bedrooms'] = dt['total_bedrooms'] / dt['total_rooms']
print(dt['bedrooms'].head())

In [ ]:
dt['people_per_house'] = dt['population'] / dt['households']
dt['people_per_house'].head()

In [ ]:
atributes = ['median_house_value', 'median_income', 'bedrooms',
             'people_per_house', 'rooms_per_house']

scatter_matrix(dt[atributes], figsize=(10,10))
plt.show()

In [ ]:
corr_matrix2 = dt.corr(numeric_only=True)
corr_matrix2

In [ ]:
corr_matrix2['median_house_value'].sort_values(ascending = False)

#### Conclusión de correlaciones

1. **Correlaciones Positivas Fuertes**:
   - **Ingreso Mediano**: Tiene una correlación positiva fuerte de 0.688075 con el valor mediano de la vivienda. Esto sugiere que a medida que aumenta el ingreso mediano, tiende a aumentar también el valor mediano de la vivienda.
   - **Habitaciones por Casa**: Muestra una correlación positiva moderada de 0.151948 con el valor mediano de la vivienda. Esto indica que a medida que aumenta el número de habitaciones por hogar, tiende a aumentar también el valor mediano de la vivienda.
   - **Total de Habitaciones** y **Edad Mediana de la Vivienda**: Ambos tienen correlaciones positivas más débiles con el valor mediano de la vivienda, lo que indica menos influencia en comparación con el ingreso mediano y las habitaciones por casa.

2. **Correlaciones Negativas Débiles**:
   - **Dormitorios** y **Personas por Casa**: Ambos muestran correlaciones negativas débiles con el valor mediano de la vivienda. Esto implica que a medida que aumenta la proporción de dormitorios en una casa o el número de personas por casa, tiende a disminuir ligeramente el valor mediano de la vivienda.

Basándonos en este análisis, podrías inferir que factores como el ingreso y el número de habitaciones por casa tienen una influencia más fuerte en el valor mediano de la vivienda en comparación con factores como la latitud o el número de dormitorios.

## Clase 4

### Preprocesamiento de datos faltantes



```
dt2.dropna(subnet = ['total_bedrooms'], inplace = True)

dt2.drop('total_bedrooms', axis = 1)

media_data = dt2['total_bedrooms'].median()

dt2['total_bedrooms'].fillna(media_data, inplace = True)

```

In [ ]:
dt2 = stratified_train_set.drop('median_house_value', axis = 1)
dt2_labels = stratified_train_set['median_house_value'].copy()

In [ ]:
null_rows_idx = dt2.isnull().any(axis = 1)
dt2.loc[null_rows_idx]

In [ ]:
datadropna = dt2.copy()
datadropna.dropna(subset=["total_bedrooms"], inplace=True)
datadropna.loc[null_rows_idx]

In [ ]:
datadropna.info()

In [ ]:
datadrop = dt2.copy()
datadrop.drop("total_bedrooms", axis=1, inplace=True)
datadrop.loc[null_rows_idx]

In [ ]:
datadrop.info()

In [ ]:
datafill = dt2.copy()
median_data = datafill["total_bedrooms"].median()
datafill["total_bedrooms"].fillna(median_data, inplace=True)
datafill.loc[null_rows_idx]

In [ ]:
datafill.info()

### Uso de sklearn

In [ ]:
imputer = SimpleImputer(strategy='median')
imputer

In [ ]:
dt_num = dt2.select_dtypes(include = [np.number])
dt_num

In [ ]:
imputer.fit(dt_num) #Imputador esta entrenado

In [ ]:
imputer.statistics_

In [ ]:
dt_num.median().values

In [ ]:
X = imputer.transform(dt_num)
X

In [ ]:
imputer.feature_names_in_

In [ ]:
df = pd.DataFrame(X, columns=dt_num.columns,
                  index = dt_num.index)

df

In [ ]:
df.loc[null_rows_idx]

In [ ]:
imputer.strategy

### Procesamiento de datos categóricos

In [ ]:
dt_cat = dt2[['ocean_proximity']]
dt_cat.head(12)

In [ ]:
ordinal_en = OrdinalEncoder()
dt_cat_oe = ordinal_en.fit_transform(dt_cat)
dt_cat_oe

In [ ]:
ordinal_en.categories_

In [ ]:
dt_cat_oe[:12]

In [ ]:
onehot_en = OneHotEncoder()
dt_cat_ohe = onehot_en.fit_transform(dt_cat)
dt_cat_ohe

In [ ]:
dt_cat_ohe.toarray()

### Conclusiones

#### Númericas



- **Dropna:** Primero, se eliminan las filas que contienen valores faltantes en 'total_bedrooms'. Esta estrategia es problemática ya que el dataset no es muy grande y hay una cantidad significativa de filas afectadas,esto resulta en la pérdida de una parte considerable de los datos.

- **Drop:** Una alternativa menos destructiva implica eliminar la columna 'total_bedrooms' por completo. Sin embargo, esta acción suprime información relevante que podría ser valiosa para el análisis posterior. Por lo tanto, esta opción puede ser adecuada solo si 'total_bedrooms' no es esencial para los objetivos de análisis, y en este caso la afirmación anterior es falsa.

- **Fillna:** Otra estrategia consiste en rellenar los valores faltantes con la mediana de la columna 'total_bedrooms'. Esto conserva todas las filas en el conjunto de datos mientras se utiliza un valor estadístico representativo para los datos faltantes. Sin embargo, es importante considerar si la mediana es la medida adecuada, especialmente si hay valores atípicos en los datos.

En conclusión, la estrategia menos destructiva implica usar el fillna para rellenar los valores faltantes con la mediana de 'total_bedrooms'. Esto preserva la mayor cantidad de información posible mientras se mitiga el impacto de los valores faltantes en el análisis de datos.

El uso del imputer (rellenador de valores faltantes) es esencial para manejar los datos faltantes en un conjunto de datos. En este caso, se utilizó el SimpleImputer con la estrategia de 'median' para rellenar los valores faltantes en las características numéricas del conjunto de datos. El estado inicial de los datos contenía valores faltantes en 'total bedrooms', lo que podría haber afectado la calidad y la eficacia de los análisis y modelos subsiguientes. Al entrenar el imputer con las características numéricas y luego aplicarlo para transformar el conjunto de datos, se rellenaron los valores faltantes utilizando la mediana de la característica.

Después de aplicar el imputer, el estado final de los datos, muestra que ya no hay valores faltantes en las características numéricas. Se han rellenado los valores faltantes con la mediana correspondiente, lo que garantiza que todas las características tengan valores válidos para su posterior análisis y modelado. Esto es crucial para mantener la integridad de los datos y garantizar la precisión y la confiabilidad de cualquier análisis o modelo construido sobre ellos.

#### Categóricas



- La aplicación de la codificación ordinal y la codificación one-hot ha tenido un impacto significativo en la representación de los datos. Con la codificación ordinal, se asignan valores numéricos a las categorías de la columna 'ocean_proximity', manteniendo así el orden relativo entre ellas. Este enfoque es útil cuando existe una jerarquía natural entre las categorías, como en el caso de datos ordinales.

    Después de aplicar la codificación ordinal a la columna 'ocean_proximity', los datos quedan representados por valores numéricos asignados a cada categoría. Por ejemplo, si las categorías son '<1H OCEAN', 'INLAND', 'NEAR BAY', 'NEAR OCEAN', y 'ISLAND', podrían ser asignadas a valores numéricos como 0, 1, 2, 3 y 4 respectivamente, de acuerdo con el orden establecido por la codificación ordinal. Esto crea una representación lineal de las categorías, lo que puede ser útil si existe una relación ordinal natural entre ellas.

- Por otro lado, la codificación one-hot ha generado una representación binaria de las categorías, creando columnas separadas para cada categoría y asignando un valor de 1 en la columna correspondiente a la categoría presente en cada fila, mientras que todas las demás columnas contienen valores de 0. Aunque esto aumenta la dimensionalidad de los datos, evita la suposición de un orden entre las categorías y es particularmente útil cuando no existe una jerarquía entre las categorías, como en datos nominales.

    Después de aplicar la codificación one-hot, los datos se transforman en un conjunto de columnas binarias. Cada columna representa una categoría única de la columna original 'ocean_proximity'.  Esta representación crea una matriz dispersa, donde la mayoría de los elementos son 0, lo que puede aumentar la dimensionalidad de los datos pero también permite que los modelos de aprendizaje automático no asuman ningún orden entre las categorías.